In [4]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord, Galactocentric, ICRS, Galactic
import gala.coordinates as gc
import sys, time
import pprint

## Test data

In [2]:
test_ra = 168.3700 #degrees
test_dec = 22.15167 #degrees
test_d = 233 #kpc
mu_a = -6.930000 #mas/cent
mu_d = -8.670000 #mas/cent
rv = 79.1 #km/s

mu_a_std = 3.73
mu_d_std = 3.91
rv_std = 0.6

## Python Functions to Get Positions and Velocities

In [52]:
#Constants
asec_to_rad = 206264.8 #Converts arcseconds to radians
year_to_sec = 3.155693e7 #Converts from years to seconds
pc_to_km = 3.08568e13 #Converts from pc to km
year_to_day = 365.2422 #Converts years to days
ra_lsr = 318.00425 #RA of the apex of the LSR motion (ie., l=90, b=0)     
dec_lsr = 48.329611 #DEC of the apex of the LSR motion (J2000.0)
ra_sa = 251.51 #RA of the apex of the solar peculiar motion (wrt LSR) J2000.0 
dec_sa = 10.129 #DEC of the apex of the solar peculiar motion (wrt LSR)
alpha_NGP = 192.85948 #RA in degrees of the Galactic Pole (J2000.0)
delta_NGP = 27.12825 #DEC in degrees of the Galactic Pole (J2000.0)
gc_GAN = 32.93192 #Angle along galactic equator from (l,b)=(0,0) to galactic ascending node
usun = -10.0 #U-component of Sun's velocity w.r.t. LSR; Dehnen & Binney                  
vsun = 11.0 #V-component of Sun's velocity w.r.t. LSR; 1998, MNRAS, 
wsun = 7.0 #W-component of Sun's velocity w.r.t. LSR; 298, 387
vlsr = 237.0 #Circular velocity of LSR w.r.t. Galaxy center in km/s
dlsr = 8.2 #Distance of LSR from the Galaxy center in pc

def sin_deg(x):
    #Sine function for degree-valued args
    deg_to_rad = 1.74532925199e-02
    return np.sin(np.deg2rad(x))
def cos_deg(x):
    #Cosine function for degree-valued args
    return np.cos(np.deg2rad(x))
def asin_deg(x):
    #Gives asin in degrees
    return np.rad2deg(np.arcsin(x))
def acos_deg(x):
    #Gives acos in degrees
    return np.rad2deg(np.arccos(x))
def atan2_deg(y,x):
    #Gives atan2 in degrees
    return np.rad2deg(np.arctan2(y,x))

def radec_to_lb(ra, dec):
    #Returns dictionary of galactic coords (l, b) in degrees, given an object's RA/DEC in degrees
    coords = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    galactic = coords.galactic
    return galactic.l.deg, galactic.b.deg

def lb_to_galactic_xyz(l, b, d, dlsr=dlsr):
    #Takes l and b in degrees, d (distance to object) in kpc, dlsr in kpc
    x = (dlsr-d*cos_deg(b)*cos_deg(l))
    y = -d*cos_deg(b)*sin_deg(l)
    z = d*sin_deg(b)
    return x, y, z #kpc

def pm_radec_to_lb(ra, dec, dist, mu_a, mu_d, rv):
    #Takes ra/dec in degrees, distance in pc, mu_a/mu_d in mas/cent, rv in km/s. Returns mu_l/mu_b in mas/cent
    c = SkyCoord(ra=ra*u.deg, dec=dec*u.deg, distance=dist*u.pc)
    pm = [mu_a, mu_d] * u.mas/(100*u.yr) 
    
    result = gc.pm_icrs_to_gal(c, pm).to(u.mas/(u.yr))*u.yr/u.mas
    mu_l = float(result[0]*100)
    mu_b = float(result[1]*100)
    
    return mu_l, mu_b #mas/cent

def pm_lb_to_galactic_vt(mu_a,mu_d,d,asec_to_rad=asec_to_rad,year_to_sec=year_to_sec,pc_to_km=pc_to_km):
    #mul, mub in mas/cent, distance in kpc
    #Converts l,b pm into tangential velocity in l-direction and b-direction,
    #and gets magnitude vt of tangential velocity
    
    #Get into arcsec/year
    mu_a = (mu_a*(1e-3)/100)
    mu_d = (mu_d*(1e-3)/100)

    vtl=mu_a*d*1000*pc_to_km/(year_to_sec*asec_to_rad)
    vtb=mu_d*d*1000*pc_to_km/(year_to_sec*asec_to_rad)
    vt=np.sqrt(vtl*vtl+vtb*vtb)
    return vtl, vtb, vt  

def galactic_vt_to_v_wrt_lsr(l, b, vtl, vtb, v_r_sun=rv, usun=usun, vsun=vsun, wsun=wsun):
    #Finds velocity of galaxy wrt LSR. velocity inputs are in km/s
    #Theta direction
    vg = v_r_sun*cos_deg(b)*sin_deg(l) - vtb*sin_deg(b)*sin_deg(l)+vtl*cos_deg(l) + vsun
    #Pi direction
    ug = -v_r_sun*cos_deg(b)*cos_deg(l) + vtb*sin_deg(b)*cos_deg(l) + vtl*sin_deg(l) + usun
    #Z direction
    wg =  v_r_sun*sin_deg(b) + vtb*cos_deg(b) + wsun
    
    return ug, vg, wg #km/s

def v_uvg_to_cyl(l,b,d,ug,vg,wg, dlsr=dlsr, vlsr=vlsr):
    #input velocities in km/s, distances in kpc
    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    sinalpha=d*cos_deg(b)*sin_deg(l)/x
    cosalpha=(x*x+dlsr*dlsr-d*d*cos_deg(b)*cos_deg(b))/(2.0*x*dlsr)

    vpi =  ug*cosalpha + (vg+vlsr)*sinalpha
    vtheta = -ug*sinalpha + (vg+vlsr)*cosalpha
    vz =  wg
    return vpi, vtheta, vz #km/s

def v_cyl_to_xyz(l, b, d, vpi, vth, vz, dlsr=dlsr):
    #distances in kpc, vels in km/s

    x=dlsr-d*cos_deg(b)*cos_deg(l)
    y=-d*cos_deg(b)*sin_deg(l)
    z=d*sin_deg(b)

    costh=x/np.sqrt(x**2+y**2)
    sinth=-y/np.sqrt(x**2+y**2)
    vx=vpi*costh-vth*sinth
    vy=-vpi*sinth-vth*costh
    return vx, vy, vz

def get_v_spherical(l, b, d, vpi, vtheta, vz, dlsr=dlsr):

    x=np.sqrt(dlsr*dlsr+d*d*cos_deg(b)*cos_deg(b)-2.0*dlsr*d*cos_deg(b)*cos_deg(l))
    cosbeta=x/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    sinbeta=d*sin_deg(b)/np.sqrt(x*x+d*d*sin_deg(b)*sin_deg(b))
    vr =  vpi*cosbeta + vz*sinbeta
    vrot = vtheta
    v_perp_radial = -vpi*sinbeta + vz*cosbeta
    vt = np.sqrt(vrot**2+v_perp_radial**2)
    
    return vr, vrot, v_perp_radial, vt

def get_vxvyvz(ra, dec, distance, mu_a, mu_d, rv):
    l, b = radec_to_lb(ra, dec)
    x, y, z = lb_to_galactic_xyz(l, b, distance)
    
    mu_l, mu_b = pm_radec_to_lb(ra, dec, distance, mu_a, mu_d, rv)
    vtl, vtb, vt = pm_lb_to_galactic_vt(mu_l, mu_b, distance)    
    ug, vg, wg = galactic_vt_to_v_wrt_lsr(l, b, vtl, vtb, rv)
    vpi, vtheta, vz = v_uvg_to_cyl(l, b, distance, ug, vg, wg)
    vx, vy, vz = v_cyl_to_xyz(l, b, distance, vpi, vtheta, vz)
    
    return vx, vy, vz

def get_xyz(ra, dec, distance):
    l, b = radec_to_lb(ra, dec)
    x, y, z = lb_to_galactic_xyz(l, b, distance)
    return x, y, z

print get_vxvyvz(test_ra, test_dec, test_d, mu_a, mu_d, rv)
print get_xyz(test_ra, test_dec, test_d)

(41.227183073492441, -115.55907658317186, 40.498068962037358)
(77.105875011103919, 58.166385352063067, 214.84331966349205)


## Print Values

In [6]:
l, b = radec_to_lb(test_ra, test_dec)
print "l = {0}, b = {1}" .format(l,b) +" (deg)" + "\n"

x0, y0, z0 = [i for i in lb_to_galactic_xyz(l, b, test_d)]
print "Galactocentric X, Y, Z = {0}, {1}, {2}" .format(x0, y0, z0) + " (kpc)" + "\n"

mu_l, mu_b = pm_radec_to_lb(test_ra, test_dec, test_d, mu_a, mu_d, rv) #*u.yr/u.mas
print "mu_l = {0}, mu_b = {1}" .format (mu_l, mu_b) + " (mas/cent)" + "\n"

vtl, vtb, vt = pm_lb_to_galactic_vt(mu_l, mu_b, test_d)
print "v_tang in l direction = {0}, v_tang in b direction = {1}, v_tang total = {2}".format(vtl, vtb, vt) \
+ " (km/s)" + "\n"

ug, vg, wg = galactic_vt_to_v_wrt_lsr(l, b, vtl, vtb, rv)
print "Velocities of galaxy wrt LSR:"
print "u = {0}, v = {1}, w = {2}" .format(ug, vg, wg) + " (km/s)" + "\n"

vpi, vtheta, vz = v_uvg_to_cyl(l,b,test_d,ug,vg,wg)
print "Cylindrical velocities:"
print "vpi = {0}, vtheta = {1}, vz = {2}" .format(vpi, vtheta, vz) + " (km/s)" + "\n"

vx0, vy0, vz0 = v_cyl_to_xyz(l, b, test_d, vpi, vtheta, vz)
print "Galactocentric XYZ velocities:"
print "Vx = {0}, Vy = {1}, Vz = {2}" .format (vx0, vy0, vz0) + " (km/s)" + "\n"

vr, vrot, v_perp_radial, vt = get_v_spherical(l, b, test_d, vpi, vtheta, vz)
print "Radial velocity = {0}" .format(vr) + " (km/s)"
print "Velocity in direction of disk rotation = {0}" .format(vrot) + " (km/s)"
print "V up and perp to radial = {0}" .format(v_perp_radial) + " (km/s)"
print "Tangential velocity = {0}" .format(vt) + " (km/s)"

l = 220.16913737, b = 67.2312443585 (deg)

Galactocentric X, Y, Z = 77.1058750111, 58.1663853521, 214.843319663 (kpc)

mu_l = 6.17081529568, mu_b = -9.22577180982 (mas/cent)

v_tang in l direction = 68.1599852049, v_tang in b direction = -101.903628602, v_tang total = 122.597443307 (km/s)

Velocities of galaxy wrt LSR:
u = 41.2271830735, v = -121.440923417, w = 40.498068962 (km/s)

Cylindrical velocities:
vpi = -36.6806351847, vtheta = 117.081645903, vz = 40.498068962 (km/s)

Galactocentric XYZ velocities:
Vx = 41.2271830735, Vy = -115.559076583, Vz = 40.498068962 (km/s)

Radial velocity = 21.896955667 (km/s)
Velocity in direction of disk rotation = 117.081645903 (km/s)
V up and perp to radial = 50.0608222039 (km/s)
Tangential velocity = 127.334982338 (km/s)


# Orbit

In [46]:
#Constants
G = 4.498933261e-6 #Gravitational constant in kpc^3/(M_sun Gyr)
vcon = 1.0226903 #Conversion factor from kpc/Gyr to km/s (i.e. multiply by km/s to get kpc/Gyr)
d2r=1.74532925199e-2

print l, b, test_d
print x0, y0, z0
print vx0, vy0, vz0


   #      if (j.eq.1) then
    #        write (6,*) 'x0,y0,z0: ',x0,y0,z0
    #        write (6,*) 'vx0,vy0,vz0: ',vx0,vy0,vz0
     #       write (6,*) 'amx0,amy0,amz0: ',amx0,amy0,amz0
      
    #write (6,*) 'lam0, bam0: ',lam(k)/d2r,bam(k)/d2r
     #       write (6,*) ''

#Direction in l, b that the angular momentum vector points

220.16913737 67.2312443585 233
77.1058750111 58.1663853521 214.843319663
41.2271830735 -115.559076583 40.498068962


## XENERGY, XFORCE

In [10]:
def gammq(a,x):
    #a-1 is power inside the integral
    #x = lower bound of integration
    #mpmath.gammainc defines them differently,
    #so we make the switch when calling it herein
    import mpmath
    #mpmath.mp.dps = 15 # arbitrary precision!
    return float(mpmath.gammainc(z=a, a=x, regularized=True))


def XENERGY(x, y, z, vx, vy, vz, vcon=vcon, G=G):
    #bulge parameters
    mb=9.1e9
    rc=2.1
    #disk parameters
    #increase disk mass to get higher v_c at R=R_0.
    a=4.9e0
    b=0.30e0
    md=1.35e11
    #halo parameters
    mv=1.16e12
    c=10.0
    ra=28.2

    #The kinetic energy per unit mass
    ke=vcon*vcon*(vx*vx+vy*vy+vz*vz)/2.0
    
    #The potential energy per unit mass
    r  = np.sqrt(x*x+y*y+z*z)
    rd2 = x*x+y*y
    
    #disk
    phi1=-G*md/np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2)

    #bulge; this isn't right, but is accurate for r >> rc and that
    #is the region of interest for all or almost all of our range of
    #integration
    
    if r > rc/2.0:
        phi2 = -G*mb/r
    else:
        phi2 = -G*2.0*mb/rc
        
    #halo
    phi3 = -G*(mv/(np.log(1.0+c)-c/(1.0+c))) * np.log(1.0+r/ra)/r

    e=ke+phi1+phi2+phi3

    return ke,phi1,phi2,phi3,e

def XFORCE(x, y, z, G=G):
    #bulge parameters
    mb=9.1e9
    rc=2.1
    #increase disk mass to get higher v_c at R=R_0.
    a=4.9
    b=0.30
    md=1.35e11
    #halo parameters
    mv=1.16e12
    c=10.0
    ra=28.2
    
    r  = np.sqrt(x*x+y*y+z*z)
    rd2 = x*x+y*y
    
    #bulge radial force: -GM_b(r)/r*r
    mb_r = mb*(1.-1.0891*np.exp(-r/rc)*(r/rc)**.2 - gammq(.2, r/rc))
    #mb_r=mb*(1.00 - 1.0891*np.exp(-r/rc)*(r/rc)**.2 - gammq(.2,r/rc))
    fb_r = -G*mb_r/(r*r)

    #NFW halo radial force: -GM_nfw(r)/r*r
    mnfw_r=mv*(np.log(1.0+r/ra)-r/(ra+r))/(np.log(1.0+c)-c/(1.0+c))
    fh_r = -G*mnfw_r/(r*r)

    #Find the X components of the force.
    fx1=-G*md*x/(np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2))**3
    fx2=x*fb_r/r
    fx3=x*fh_r/r
    fx=fx1+fx2+fx3

    #Find the Y components of the force.
    fy1=-G*md*y/(np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2))**3
    fy2=y*fb_r/r
    fy3=y*fh_r/r
    fy=fy1+fy2+fy3
    
    #Find the Z components of the force.
    fz1=-G*md*z/(np.sqrt(rd2+(a+np.sqrt(z*z+b*b))**2))**3
    fz1=fz1*(1.0+a/np.sqrt(z*z+b*b))
    fz2=z*fb_r/r
    fz3=z*fh_r/r
    fz=fz1+fz2+fz3

    return fx, fy, fz

print XFORCE(x0, y0, z0)

ke, phi1, phi2, phi3, e = XENERGY(x0, y0, z0, vx0, vy0, vz0)
print ke, phi1, phi2, phi3, e

(-31.392332139809479, -23.681444352731305, -87.684957932643584)
8729.91904415 -2530.30156307 -173.803322552 -33269.8254141 -27244.0112556


In [11]:
#Create a dictionary of force funtions. This gives an easy way to get
#fx, fy, fz for any potential by changing a single parameter

forcefunctions = {'X':XFORCE}
#Test it
fx, fy, fz = forcefunctions['X'](x0, y0, z0)
print fx, fy, fz #It worked!

energyfunctions = {'X':XENERGY}

-31.3923321398 -23.6814443527 -87.6849579326


## Construct sample of proper motions

# Integrate orbit

In [108]:
#ACTUAL NSTEP = 500000 #Number of integration steps
NSTEP = 500
t = 100 #total integration time in Gyr
dt = float(t)/float(NSTEP) #timestep interval size in GYr

def integrate_orbit(pot, x0, y0, z0, vx0, vy0, vz0, vcon=vcon, t=t, NSTEP=NSTEP):
    dt = float(t)/float(NSTEP)
    timestep = 0.0
    times = [timestep]
    position = (x0, y0, z0, x0*x0+y0*y0+z0*z0)
    positions = [position]

    #Initial angular momenta
    amx0=y0*vz0-z0*vy0
    amy0=z0*vx0-x0*vz0
    amz0=x0*vy0-y0*vx0
    am0=np.sqrt(amx0*amx0+amy0*amy0+amz0*amz0)
    bam0=np.arcsin(amz0/am0)
    lam0=np.arctan2(-amy0,-amx0)
    if(lam0 < 0.):
        lam0 += 2.*np.pi
    
   # position = [(x+dt*vx*vcon, y+dt*vy*vcon, z+dt*vz*vcon)
    x = x0
    y = y0
    z = z0
    vx = vx0
    vy = vy0
    vz = vz0
    #fx, fy, fz = forcefunctions['X'](x0,y0,z0)
    for i in np.arange(0, NSTEP):
        fx, fy, fz = forcefunctions['X'](x, y, z)
        vx=vx+0.5*dt*fx/vcon
        vy=vy+0.5*dt*fy/vcon
        vz=vz+0.5*dt*fz/vcon
        x=x+dt*vx*vcon
        y=y+dt*vy*vcon
        z=z+dt*vz*vcon
        r2 = x*x+y*y+z*z
        timestep = timestep + dt
        times.append(timestep)
        positions.append((x,y,z,r2))
        sys.stdout.flush()
        sys.stdout.write('\r' + 'Done {0}' .format(i))      
    print

    #Final energies
    finalenergies = energyfunctions['X'](x, y, z, vx, vy, vz)
    
    #Final angular momenta
    amx=y*vz-z*vy
    amy=z*vx-x*vz
    amz=x*vy-y*vx
    am=np.sqrt(amx*amx+amy*amy+amz*amz)
    bamf = np.arcsin(amz/am)
    dbam = bamf - bam0
    lamf=np.arctan2(-amy,-amx)
    if lamf < 0.:
        lamf += 2.*np.pi
        #check to make sure have not crossed 0/2*pi
    if np.abs(lamf-lam0) > np.pi:
        if lam < np.pi/2.:
             lamf -= 2.*np.pi
        else:
             lamf += 2.*np.pi
    dlam=lamf-lam0
        
    #return positions, finalenergies, peri, apo
    return {'positions':np.array(positions), 
            'finalenergies':np.array(finalenergies), 
            'times':np.array(times), 
            'lam0':np.rad2deg(lam0),
            'bam0':np.rad2deg(bam0),
            'dlam':np.rad2deg(dlam),
            'dbam':np.rad2deg(dbam)}

## Analyze Orbit to get Peri, Apo, Ecc, Period

In [23]:
result_test_idorb = integrate_orbit('X', vx=vx0, vy=vy0, vz=vz0)
test_r2=[position[3] for position in result_test_idorb['positions']]
times = result_test_idorb['times']

Done 399


In [24]:
test_x = [position[0] for position in result_test_idorb['positions']]
test_y = [position[1] for position in result_test_idorb['positions']]
test_z = [position[2] for position in result_test_idorb['positions']]

In [91]:
def find_apo_peri(r2):
    id_pe = [i for i in np.arange(1, len(r2)-1) if r2[i-1] > r2[i] and r2[i+1] > r2[i]]
    id_ap = [i for i in np.arange(1, len(r2)-1) if r2[i-1] < r2[i] and r2[i+1] < r2[i]]
    #N_pe = len(id_pe)
    #N_ap = len(id_ap)
    
    apos = [r2[i] for i in id_ap]
    peris = [r2[i] for i in id_pe]
    apo_mean = np.sqrt(np.mean(apos))
    peri_mean = np.sqrt(np.mean(peris))

    return id_pe, id_ap, peri_mean, apo_mean

id_pe, id_ap, peri_mean, apo_mean = find_apo_peri(test_r2)

def find_ecc(r2, id_pe, id_ap):
    N_pe = len(id_pe)
    N_ap = len(id_ap)
    eccentricities = []
    for i in np.arange(0, min(N_pe,N_ap)):   
        e = (np.sqrt(r2[id_ap[i]]) - np.sqrt(r2[id_pe[i]]))/(np.sqrt(r2[id_ap[i]])+np.sqrt(r2[id_pe[i]]))
        eccentricities.append(e)
    return np.mean(eccentricities)

def find_period(times, id_ap): 
    #Find average period from R_a to R_a 
    periods = []
    for i in np.arange(1, len(id_ap)):
        period = times[id_ap[i]]-times[id_ap[i-1]]
        periods.append(period)
    return np.mean(periods) #Gyr

def pick3points(idpe,idap,index,x,y,z):
    #x, y, z are arrays, i.e. time series of x, y, z values
    if idpe[index] < idap[index]:
        #perigalacticon occurs first
        #Peri first
        point1 = (x[idpe[index]], y[idpe[index]], z[idpe[index]])
        
        #Apogalacticon last
        point3 = (x[idap[index]], y[idap[index]], z[idap[index]])
        
    else:
        #apogalacticon occurs first
        #Apogalacticon first
        point1 = (x[idap[index]], y[idap[index]], z[idap[index]])
        
        #Perigalacticon last
        point3 = (x[idpe[index]], y[idpe[index]], z[idpe[index]])

    #Point between the two
    indexbetween = (idpe[index]+idap[index])/2
    point2 = (x[indexbetween], y[indexbetween], z[indexbetween])

    return [point1, point2, point3]

def FINDPLANE(points): #points is list of three 3-tuples
    point1, point2, point3 = points
    
    #Vector p1p2 goes from p1 to p2, vector p1p3 goes from p1 to p3
    #The points are ordered in time, so galaxy reaches p1, then p2, then p3
    #Taking coss product p1p2 X p1p3 thus results in a normal vector to 
    #orbital plane, this vector being parallel to the angular velocity vector, 
    #by right hand rule
    
    p1p2 = (point2[0]-point1[0], point2[1]-point1[1], point2[2]-point1[2]) 
    p1p3 = (point3[0]-point1[0], point3[1]-point1[1], point3[2]-point1[2])

    return np.cross(p1p2, p1p3) #Normal vector

def FINDINC(normalvector):
    #Normal vector is assumed to be parallel to angular velocity vector
    a, b, c = normalvector 
    
    #Inclination angle of orbit plane to galactic plane.
    #Zero is in galactic plane and orbiting in the direction of galactic rotation
    theta=np.arccos(-c/np.sqrt(a*a+b*b+c*c))
    return theta #radians!

def FINDLONGITUDE(normalvector):     
    #longitude of ascending node measured in direction opposite galactic rotation from x=0
    a = normalvector[0]
    b = normalvector[1]
    omega=np.arctan2(b,a)+np.pi/2.0
    if omega < 0:
        omega += 2.*np.pi
    return omega

def find_lpole_bpole(theta, omega):
    bpole=theta-(np.pi/2.)
    lpole=omega+0.5*np.pi
    lpole = np.rad2deg(lpole)
    bpole = np.rad2deg(bpole)
    #if lpole > 2.*np.pi:
    #    lpole -= 2.*np.pi
    if lpole >= 0:
        lpole = lpole % 360
    else:
        lpole = lpole % -360
    if bpole >= 0:
        bpole = bpole % 360
    else:
        bpole = bpole % -360

    return lpole, bpole

In [113]:
    #do some checking to avoid splitting omega values across 0/2pi
    #and the same for lam
    #     if (abs(lam(k)-lam(1)).gt.pi) then
    #        if (lam(1).lt.pi) then
    #           lam(k)=lam(k)-2.0d0*pi
    #        else
    #           lam(k)=lam(k)+2.0d0*pi
    #        end if
    #     end if

(192.27907403048422, -22.123000968319303)


(192.27775500506823, -22.131565669277787)

In [28]:
#NOT YET WORKING

ldsph = 1.e3
mol1 = 1.
mol2 = 3.
dg = 233. #kpc
ratoam = 3437.747

def get_rt(r2, idpe, idap, ldsph=ldsph, mol1=mol1, mol2=mol2, dg=dg, ratoam=ratoam):
    #Calculate the tidal radius at pericenter and store it
    N_pe = len(idpe)
    N_ap = len(idap)
    rt1 = []
    rt2 = []
    for i in np.arange(1, min(N_pe, N_ap)):
        r_ap = np.sqrt(r2[idap[i]])    
        r_pe=np.sqrt(r2[idpe[i]])
        ec=(r_ap-r_pe)/(r_ap+r_pe)
        fe=((1.-ec)**2)/((((1.+ec)**2)/(2.*ec))*np.log((1.+ec)/(1.-ec))+1.)
        mmw=1.1e10*(r_ap+r_pe)/2.
        rt1.append(((r_ap+r_pe)/2.)*(fe*mol1*ldsph/mmw)**0.333333)
        rt2.append(((r_ap+r_pe)/2.)*(fe*mol2*ldsph/mmw)**0.333333)
        
    rt1=np.mean(rt1)
    rt2=np.mean(rt2)
    rt1=np.arctan(rt1/dg)*ratoam
    rt2=np.arctan(rt2/dg)*ratoam
    
    return rt1, rt2
    
rt1, rt2 = get_rt(test_r2, id_pe, id_ap)
rt1 = np.rad2deg(rt1)
rt2 = np.rad2deg(rt2)
print rt1, rt2

NameError: name 'id_pe' is not defined

## Putting together the MC Sample Construction + Integration + Parameters

In [114]:
NMC = 1 #Number of MC experiments. Set back to 1000 later

#Get XYZ from RA, DEC
x0, y0, z0 = get_xyz(test_ra, test_dec, test_d)

print "Constructing sample of {0} proper motions from Gaussian distributions of mu_a, mu_d, radial velocity..." \
.format(NMC)
mu_a_sample = np.random.normal(loc=mu_a, scale=mu_a_std, size=NMC-1)
mu_d_sample = np.random.normal(loc=mu_d, scale=mu_d_std, size=NMC-1)
rv_sample = np.random.normal(loc=rv, scale=rv_std, size=NMC-1)

pm_sample = np.column_stack([mu_a_sample, mu_d_sample, rv_sample])
pm_sample = np.insert(pm_sample, 0, (mu_a, mu_d, rv), axis=0) 
#Now the first value is a tuple with the measured or "actual" mu_a, mu_d, rv

print "Converting to sample of xyz velocities..."
velsample = [get_vxvyvz(test_ra, test_dec, test_d, pm[0], pm[1], pm[2]) for pm in pm_sample]

#Make bunch of empty lists for our orbital parameters
apos, peris, es, periods, thetas, omegas, lpoles, bpoles, lam0s, bam0s, lamfs, bamfs = [[] for i in np.arange(12)]

for velocityvector in velsample:
    vx0, vy0, vz0 = velocityvector
    result = integrate_orbit('X', x0, y0, z0, vx0, vy0, vz0, t=100, NSTEP=500)
    positions = result['positions']
    times = result['times']
    x = [pos[0] for pos in positions]
    y = [pos[1] for pos in positions]
    z = [pos[2] for pos in positions]
    r2 = [pos[3] for pos in positions]

    #Get apo and peri
    id_pe, id_ap, peri_mean, apo_mean = find_apo_peri(r2)
    N_pe = len(id_pe)
    N_ap = len(id_ap)
    print peri_mean, apo_mean
    apos.append(apo_mean)
    peris.append(peri_mean)
    
    #Get eccentr
    ecc = find_ecc(r2, id_pe, id_ap)
    print ecc
    es.append(ecc)
    
    #Get period
    avg_period = find_period(times, id_ap)
    print avg_period
    periods.append(avg_period)
    
    #Get Inclination and Longitude
    setsofpoints = [pick3points(id_pe, id_ap, i, x, y, z) for i in np.arange(1,min(N_pe,N_ap))]
    inclinations = [FINDINC(FINDPLANE(pointset)) for pointset in setsofpoints]
    longitudes = [FINDLONGITUDE(FINDPLANE(pointset)) for pointset in setsofpoints]
    avg_theta = np.mean(inclinations)
    avg_omega = np.mean(longitudes)
    thetas.append(np.rad2deg(avg_theta))
    omegas.append(np.rad2deg(avg_omega))
    print np.rad2deg(avg_theta), np.rad2deg(avg_omega)
    
    #SAEM THING AS ABOVE
    #for i in np.arange(1,min(N_pe,N_ap)):
    #points = pick3points(id_pe, id_ap, i, test_x, test_y, test_z)
    #normalvector = FINDPLANE(points)       
    #theta = FINDINC(normalvector)
    #omega = FINDLONGITUDE(normalvector)
    #print normalvector, theta, omega
    
    #Get lpole, bpole
    lpole, bpole = find_lpole_bpole(avg_theta, avg_omega)
    lpoles.append(lpole)
    bpoles.append(bpole)
    print lpole, bpole
    
    #Get initial angular momentum
    lam0 = result['lam0']
    bam0 = result['bam0']
    lam0s.append(lam0)
    bam0s.append(bam0)
    print lam0, bam0
    
    #Get rt1, rt2
    rt1, rt2 = get_rt(r2, id_pe, id_ap) 
    print rt1, rt2
    
    #dlam
    dlam = result['dlam']
    print "dlam =", dlam
    
print 
print np.nanmean(peris), np.nanmean(apos)
print np.nanmean(es)
print np.nanmean(periods)

Constructing sample of 1 proper motions from Gaussian distributions of mu_a, mu_d, radial velocity...
Converting to sample of xyz velocities...
Done 499
229.673888411 459.021817299
0.333017809466
18.55
67.8684343307 102.277755005
192.277755005 -22.1315656693
191.913012306 -22.1488456026
1.75532590909 2.53161687342
dlam = 0.622682101852

229.673888411 459.021817299
0.333017809466
18.55


## Deprecated

## Get Galactocentric tangential velocity components

In [ ]:
#Upper incomplete gamma function

#def gammq2(a, x):
#    import scipy.special
#    #Upper incomplete gamma function i.e. 1 - gamma
#    #return scipy.special.gammaincc(a,x)
#    return scipy.special.gammaincc(a,x)
#print gammq2(1,2)